In [1]:
'''
Reference:
Example script from Keras Github Repository keras/examples/lstm_text_generation.py
https://github.com/keras-team/keras

Same configuration as the vanilla LSTM char-RNN for Donna Summer, except with 512 hidden units
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#text = io.open(path, encoding='utf-8').read().lower()
import pandas as pd
lyrics_data = pd.read_csv('songdata.csv')

# Donna Summer

lyrics_data_donna = lyrics_data[lyrics_data.artist == 'Donna Summer']['text']

lyrics_concat_donna = u''
for i in lyrics_data_donna.index:
    lyrics_concat_donna += lyrics_data_donna[i]
    
print("input chars length")
print(len(lyrics_concat_donna))
text = lyrics_concat_donna
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, input_shape=(maxlen, len(chars))))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Using TensorFlow backend.


input chars length
236908
corpus length: 236908
total chars: 73
nb sequences: 78956
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
78956/78956 [==============================] - 48s - loss: 4.4033    

----- diversity: 0.2
----- Generating with seed: "ws  
  
They're just cats without claws "
ws  
  
They're just cats without claws v vvv v v g vvv vgv vv v vvv v  v v v    v     v  v vv  v  v vv  vvvvv    v   g   v vvvv tvv vvvvvg vv         vvv v  vv v   gvv    v v vv  vv     v    vv  g vv    vv    vt    vvv  vvv  v v g vvv vv  v  vv  vvvv v   v vvv v        vv v vv vv v  v   v v  v vv gv     vv v vv v vvv v    vv  vvv vv vvv  vvvvv g  g v v vv v   vv v v  v    vvvv      vgv  v v v   v v    vvv vvv   vvv vvv  gg  vv  v v v v

----- diversity: 0.5
----- Generating with seed: "ws  
  
They're just cats without claws "
ws  
  
They're just cats without claws  gvgg  vgvvgvgv g v h     v u  vggg  gvuvvv vvvv  vgu ggv gd s    t vvtv 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


                                                                                                                                                                                                                                                                                                                                                                                                   

----- diversity: 0.5
----- Generating with seed: "When you've never been out on your own  "
When you've never been out on your own  e   e e  e e     k        e e   ee              e    e ee     e k                te   e    e e  e   k     e  e k e    e          e                 et        e         e e  e   
      e          
    e   k            e                  e          e               k  e      e        ee         e              t                         i    e                      k e    e     ee                       e

----- diversity: 1.0
----- Generating with seed: "When you've never been out 

Still I took hir                                          r            r                                                        r      n               n                e    r          e      r                     r                                 r   r                 r        n    h r              r                                r n                    n            r         r     r            n           r r r

----- diversity: 0.5
----- Generating with seed: "id everything I wanted  
Still I took hi"
id everything I wanted  
Still I took hir ei r  ynr     rarysrar   
r rs   r n  e rrr ri  rr 
 e rr i n
 
  rne  rh iina rrr 
rre   hhh     s  rsrrar i hr hrrs snuei y r

y   an 


risrr s rsi r y r drarna  y rr   r          n
nr s
 rn    ner a
  en yarenre a ra i  er rirsnrir   A n
 n 
 l
n
   
 r y  i nhry rh
hcr   aerrrrrr e  si  rrn arii  n i  nnsn  irr s  
r i  
 er e rr    rir   ri s   i  
r
 r nrnvyrs   aa rar  i  rhr r   ri r  y

----- diversity: 1.0
----- Generating with seed: 

...

If only in my tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt

----- diversity: 0.5
----- Generating with seed: "l be home for Christmas  
If only in my "
l be home for Christmas  
If only in my tttttttttttttttttttttttttttttttttttttttttttttttattttltttttttttttttttttttttttttttttttttttttttttttttttettttttetttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttuttttttttttttttttttttttttttttttttttttttttttttttttttttttttttettttttttttttttttttttttttttttotttttttttttttttt

----- diversity: 1.0
----- Generating with seed: "

To hold me, to scold        g           g        g          g             g           gg       k                                               k                           g   g                             g            g                                                                                                                   ns                             g                             g                 g      

----- diversity: 0.5
----- Generating with seed: "e me, to guide me  
To hold me, to scold"
e me, to guide me  
To hold me, to scold go ken   s   gsg      s        ggkgg n   g  gs    g  n  og gln gs  o g   g     g     gg    g kn  gg g  g e   gn o g    gg  y  s       ggg n sk n     g  n     n   ss   n gn   na gs sggo g       gn l     k        n nk      n sgnkss g    e    
nsgg       g   l   g   g s n  gs l  g      g  g  g       a    gag  gg  n     nlo    g k g l  sg   ggn        gn g gne gsnk   gy  g n  n   nl s  n   n  l    n 

----- diversity: 1.0
----- Generating with s

He doesn't evepppppppppppppppppppppppppppp p pppppppppppppppppppppppppp ppppp pp pppp pppppppppppppppp p pppppppppp ppppppppppppppppppppppp  p ppppppp pppppp ppp ppppppppppppppppppppppppppppppppp pppppppppp popppppp pppppppppppppppppppppp pppppppp  ppppeppppppppppppppppp ppppppppppppppepp pppppppppppp pp ppppp pp  pp pppp pppppppppppppppppppppppppopppppp ppppppppppppppp pppppppppppppppppppppp  ppppp ppppppppppp

----- diversity: 0.5
----- Generating with seed: "assing through my world  
He doesn't eve"
assing through my world  
He doesn't evereppppe p pe ep  e ppopss pp p pppppp psppp pplnpp  p  ep p p ppopopspp  pppp pppppppn p p  pp   nyppnpn p pppppp spopoppp p p  ppspspp p ppshpps pps  ppsiop  op pppp p pppppp pp pppp ppepppppoppofsp npppp hrppeps  ei p  ppp  ps ep ophpp o  peopnpppsp pnppp   ipppppr ppppp ppp p poppppppp ppsph pp pppepppppop pp    p   pppoorppp pppppop p pp ppp  ep
 pnrpppp p pp pppsppsppepoppppppppopppppp pppppp

----- diversity: 1.0
----- Generating with seed: "

Oh, rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrcrrr rrrrrrrrrrccrrrrcrrrcrrrrrrrrrrrrrrrrrrrrrrcrrrrrrrrrrrrrrrrrrrrrrrrrrcrrrrrrrrrrrsrrrrrrrrcrrrrrrrrrrrrrrcrrrirrrrrerrrrrrcrrrrrrrrrrrrrrrrr rcrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrcrerrrrrrrrrrrcrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrcrrrrrrrrrrrrrrrrrrrcrr

----- diversity: 1.0
----- Generating with seed: "ep inside  
Just want you to know  
Oh, "
ep inside  
Just want you to know  
Oh, c rrrcrorrerrrrssrrtorrrrrrrrrrrricrarrvrerrmcr crScorrrrrrarrrrrcrecrdtrrsrrtcpererrrrirsrr tr rerrrcerrrrsrlrrrrrcrrrrrrcrrrrrrcrccrrrrrrrrIarcrrrrtircmrrrcrrvrctrasrrcrrcrr osrircrermrrgrcp crssrrcerarrrcdc rr rrrirricrrvrsrrmtrrccrrtrcrrsa ssrrrrrulrr rrrrrrrrrrrrrrierrirshrrlrorrrrrrrm
 rrrrrrrrcscr rrrrrotrrcrrrricr crrrcrarrrrerirrmriarcrerecrvreolcrdrrrrrsrrcrnrrrrrrrrceeirirrvrr rrr
rrvrd

----- diversity: 1.2
----- Generating with seed: "ep inside 

There's a rumloeiliolselliSllslylolhollilolloelolipsytliuollouollillloollsSuslllssoleieloiiyollolllhloelllllellolisilllllviceollSouluuoesloyoltlslyllllsiioulielollllleillllellllyslolloelll
olloolllillllsiilyoiosollyeleollltolhohoslelyolliielllsui
iellil
eslseolislelollsulholssollslloolpllllloloyyiielpslrlsyslollulillltliillllillloeslliuilloloimilslsslolllillpiushdli ullluSlorllili
loslsoeoooil
lolloiosolullllv

----- diversity: 1.0
----- Generating with seed: "a rumour  
Rumour has it  
There's a rum"
a rumour  
Rumour has it  
There's a rumlyeospsloteho'slse
liiliuhudihilliiieimlhlo lmyseoetuviy
ilihitiupileooylmlhsporsSiollrselslstlhdlleuyiiomoisololprilmoSohlhillsleol
ss
iiiis
d
mlodoriohryuelhyeo
op
liuussyeshleokls lvlilel
lilmsuhSpliiooalhnlothiprlyio'dhSeilliSyehs
loseuhdlhldiloussrSuplSosuyepiSlloioplSoeeytt
d'hyld lshSmsrolmhl
tdusllooteolllu
lmso lheiiyllimlpeeyullslvolhopftehsl'isls'Ssuleitouh
Aorlos
llrlio pihSisohiioslpu

----- diversity: 1.2
----- Generating with seed: "a

In [2]:
# sentence 1

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Loneliness is always looking for a frien'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Loneliness is always looking for a frien"
Loneliness is always looking for a frien
ee


e
eeeee

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


eoeoe
eeee

e
eeooeeeeeee

e
eeeeee
o'eetoeeee


ee
eeeoe
e

eee
eeeeoeee
oee
eoeoeeo
eee
eeee

eeeoeeoeeeeee
e
oeeeeoee
eeeeee
eeeee
eeoeeeeeeee
ee'
e
e
eooese

eeo

e


eoee
oeeeeeeeoe
ne
oeeeee
e

oeoeo
ee


eeee
e
e
oe

eeeeeeeeoo
eaeeeeeeeeeeeoe
eeeo
ee
eeeeo

oee
eeeeeee
oeeee
aa
oee
oeee
eeee
eee
eeneeeeeeeeeeee
eeoe
eeeoeeeeeeIeeeeeeoeee
e
eeeee
eee

e
eee
e

eee

e'e
ee

e
eeeeee
ee

ee

eee
ee
eeeeeeoeeeeeeee
eeee
oe

eeeo
ee
e
eeeeee
e

oeoeee

eoeee

o
eeeee
aeoeeeeeoeoee
eeoeeee
e
'ee

oeoeoeo

eo
oeeo
oeoe
e
oeeeoeeoeeee
e
eeeeee
ses
eeee
neooeeeeee
oe
oee
e
ehoteoeeee
eeeeeeeeeeeeeoeee

teeeeee


'eaeeee
eeeo
eee

e
eeee
eeoe
e

e
ee
e
oeeeeee

ee
ee
eeeoeeeoee
eee

'e
ee


eee
eeo

e
eeeeeeeaeeeee

eoeeeeoeeeo

eeee'eeeoee
eoeeee
eeeee
eeoee
eeeeoeeee
eeeeooeoeee
e
e
o
tooneee
e
ee
eee
e
e'ee

eoeoeeeeeeoee
eeeeeeee
eee
eeee
e
ee
eeeeeeeetoe
ee

eeeoeee

eeee

----- diversity: 0.5
----- Generating with seed: "Loneliness is always looking for a frien"
Loneliness is alway

In [3]:
# sentence 2

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "We go hard or we go home \nWe gon' do thi"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "We go hard or we go home 
We gon' do thi"
We go hard or we go home 
We gon' do thieeee
eeoe
e

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


eneeeo
e

e
eeeeee
eeoeeeeeee
e
eeeee
e
e
oeeeoo
eeeooe

e
eeeeee
eeeeeeoooeeeo
eeehe
oeeeeeeeee
ee
ee
teeeeee
e
ee
oee
e

eeoeeeeoeaoee
eeeee
ee
eee
o

ee

eeeneeeeeo
eee
eeoeoe


o
oeeeeeeeeoe
eeee
et

eoeeee
e
e
e
ee
e
e
e
eeeoseoee
eeee
eee
ee
o

eoeo
eee

eooeee
eoe
e
eoeeeee
a'oee
eeeeeee


eeee
e
eee
eee
eeeeee
eeeeeeeee
eee
teeee
ee
e
eeeee


eoeeeeeeoeeeeee
eeeeeeoe
eee
o
eo

eoeeeeeoee
eeoeeoe
e
eeeo
eeeoe
e
eeee
ee

eee
ote

eeeeaeeeoeoee

eee
eeeee
eeeeeeeeeeeseeeeeoe

eo
eeee
e
eoooeeoee

o


eeo

o

eeeee
eeeeeeoeeee
eee
ee
eseeeeoe
eee
e
ee
eeeeeeeoe
ehe

eeeeeeeo
e
eooe

eeeeeet
e
eeoe
eoee
eeee



oeeoeeeee'e
ee
ee
ee
oeeee
eetee
e
ete


ee
eeeeeeeee
o
eee

eeeeoe
eee
eeee
eot
oeeeeeeeeee
e

ee
eeeeoee

eeeee
eooe
oaooeoeee
e
eoeeoeeeeeeeoea
eeee
oe
eee
eee
ooee
eee
e
e

oe
eeeee

oeoteo
e
eeeo
oeeeeteeeeeoe
e
eeeee'eoteeeeeeee
eeeeeeoe

ee
ooeeee
eee
e

eee



----- diversity: 0.5
----- Generating with seed: "We go hard or we go home 
We gon' do thi"
We go hard or we 

In [4]:
# sentence 3

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "At first I was afriad, I was petrified \n"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "At first I was afriad, I was petrified 
"
At first I was afriad, I was petrified 
en
ee
ee

ee

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log



eeeeeeeeee
eeeeeeeeeeoe
e
ateee
o

oeheeeotee
eIoe
eeeeeeeeeeeeeetee
oeeeeeeeeseoee
e


eeeeeeeoe

eeoe
eeeooe

eeeoeeoeeeeeeeeeoeooe

eeee
ee
ee
o


oeoee
eeeeeeee
eoeee
oeeo
eeeee

ee
eee

eee

e
eee

eee
eeeeeeeeeeeeee
eeeeee
eee
e
e
ee

eeeet
e

eeo

teeeeeeeeoeeooeeeoe
eeeeeaee
ee

eeoo
o
ee
eeee

e

eeeeeoe
eeeeeeee
eeeeoee
eeeee
eoeeeeooeeeoeeeeoeeeoeeeo
eeeee
eeooeeee
oeoeeeoee
ee
o
oe
e
e
ee
ee'


o
oeeeeeeeteeee
eeee
eaeeeeeeeeesoeeoeeeeeee
eeo
eeeeeooo
eeo
e

eoeeeee
ee
eeeee
e
eeoeoeeeeo
ee
o
eeee
o
e
eeene
eeoeeeee

eeete
e
eeeoetee



ee

eeeeeoeoee
et
eoeeeeeeeeeeoe'o
eeeeeeoeeeoe
o



ee
oe
eeeeeee
eeoeeoeeeeeee


e
eo
ee

o
eoeeoeeee
eeeeee
e

eeeeee
ee

oeee

eteeeeeee

ee
ee
e
eeeeeeeteo
ooeeoee
eeeee
eooet

eee

o
eeeoee

eeeeee
eeeeene

e
oee
o
ee

e
eee
e
eoee
ehoeeeeeeeoeeo
ee
ee
Ieaeeee
eeeeeeeeee
eeeo
e
ee
eee

eeeoe
e
eeeee
eeee
o
oeeoee'oe
eeeee
e


----- diversity: 0.5
----- Generating with seed: "At first I was afriad, I was petrified 
"
At first I was afr

In [5]:
# sentence 4

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Young Money, Nicki Minaj, Justin  \n  \nSh"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Young Money, Nicki Minaj, Justin  
  
Sh"
Young Money, Nicki Minaj, Justin  
  
Sh
ee
eeeoe
a

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


ee
eesee
eeeeeeeee
eteot
ea
eee

eo
eeo
eeee


eee
ee
eeeeee
eeeeeeeee
oee
eeee
e
etee
eee
oeee
eoteeoe
eetee
e
eoe
e'eeee

e
eeoeeee
eeoe

eeeee
eo'eeseeeteeee
eeeeeeeeeeoe


eoee'oee

oeeee
eeeeoee
ee
eeeeeee
ee


eeeeeoeeeoeoo
eo
eooeeeeoeee
eoeeeee
eeeeeot

eeee
e'o
eeeoao
eeoeeeo
e
eee
eee
e

e
ee
eee
eeoeeeeeeeeeee
ee'o
eetoeeeee

eeoeeoeeoeeeee

eoeeoeeeeee

oeeeeeo
eee
oeoeeeeeoeeo
eee
ee
eeeeeht
ee
eee
eoee'eee
eeeeeeeeeeeee

ehe
e



e
ee

oteeoeee
ete
eoeeeeeee
ee
eeeeee
e
eeeoeeee
soeeee

oooeee
ee

e
eeeoeee

eeeeeoe
eeeoee
ee
eeoeeoee
oeoeoeeeeeeoeeoeoee
eeeeee
e
eeeeeee
oeee
eeee

eeeeeee

e

oeee
eeeoeee
eeee
ee

eeeeeeeee
ee
oeeeeeeooeee
eeeeee
e
tee


eeeeeeeeeootee'eeeeee
e
eeeeoeeeeeee
eee
eee

oee
e
ee
eee'oee



eoeeeeeeete

ee
eeee

t

ee
eeoeeee
eee
eeo
o
eoeeeeeeeaeeoe
o
e
oee
eeee
e

eeeeeeeeeeeoeeeo

ee
eee
eeeeeeeee
se

eee
e
eeeeee
eeeeoee
oeoeoete

----- diversity: 0.5
----- Generating with seed: "Young Money, Nicki Minaj, Justin  
  
Sh"
Young Money, Nick

In [6]:
# sentence 5

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Life is to be lived on the positive tip "
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Life is to be lived on the positive tip "
Life is to be lived on the positive tip eeoee
eeeoee

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


eoeeeee

eeeo
eeoese
e

ee


e
eeeeet
e
s
o
e
ee

o
e
e
eeeeee
teeee

e

eeo
eeeoeeeee
e
ee
eeoeoaeeeee
ee
e

eee
eee
eee
eeeot
e
e
eoe
ee
eeeee
e


e
heeeeeeeo
eo
eeee
oooe

eeeeee
eeoe
oeo
eeeeeee

eeeeeeo
eeeet

eeoo'e
e
eeeeeeeee
eeee

e
oeeeeeeeee
e
o
eeeteeeeeee
eo
e
e
eeeeeeeeeeee
eeeeeeeteoeehe
o
e
ee'e
e
e


e
eeeteeee
eeeeeeeoeeeeea
eeeeeeeoeoeeete'

eoeee

eoee'eeeoe


e
e
oeeeeoeeeeeoe

eeeee


oeeeoeoeeeee
e
e
e


eeoe
eooeeoeee
aeooe
eeeeeeee
e

'eeee
eoeee
e
eeeeoe
eeee
eeeeeee
e
eee
e

seeeeeeeeeeeeet



ee


eeoeoe
e
o
eeeeeeeeeeeee
oeee



ee

o

eee
eeeeee


eoeeehoe

eeeooeee
eoe
eee

eee
eeo

e

eeo
oee
eoeeeeeeet
eeoeeeoeeo
n
e
e

e
ee
eee
oeeeeteo
eee'eeeoeeee
eeeeeee
oeeeoee

e
e
eeeeeeeoeeeoeo'eeeee
eeeeeeoeeoeeeeet
eee
heee
eoeee
ee
eeeteeeeee
oeeeoo
ee
eeee
eoee
eeeeeeeoseeeo
eeeeeeeee
eeee
eetaoeo
eeeeo
eeeeee

e
eoeoee
ee
eee
eeeoee
oeee
oeeeeee
t

----- diversity: 0.5
----- Generating with seed: "Life is to be lived on the positive tip "
Life is to be live

In [7]:
# sentence 6

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Another night I'm waiting for you  \nWait"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Another night I'm waiting for you  
Wait"
Another night I'm waiting for you  
Waitoeo
eeeeee
e

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


ee
teeee
eooee
eo
e
ooeeeeeeeo
e
'
eeooo
eeeeeoe
ee

o
ee
eeeeoeeeo
eee
eeeeeoeee

eeee
eeeoe
eoeeee
e
eo'e

on
ee
e
eeeeeeeeeto

oeeooe
eoeee

ee
eeoeoee

ee


oee

e'e
oeeeeeoeeeeeeeteeeeeo
e
tote
eee'o

e



o
ee
e
e
oeoooo
eeee
eeeeeeeeeoee
eeeeeeeeaoeeaeee
oe
e
eeee
eeee

oeo

eeoeeo
eeeeeeee
eoeeoee
oeo
eeeeeeee
eeeeee'e
eeeoeoeeeee
oe
eee
teeee
te
eooeete

eeee
e

eeeeteteeeo
oeee

eo
eeeoeoe
e

e
o
eeeeeeooee
e
eeeeeeeeeeeee
oeeeeeeoee
eee
eeeoe
t
e
ee
oe



eeoeoeo
e
eo'eee
ee
eeeeee
eee
e

ee
e
eeeee
eeehes
tee

oeeoeee
eee

eeee
eoeeeee
eeoee
eoote
eeee
eee
e
eeeoeeee
eeoeeee
o

eee
eo
eo
eeo
eee
eee
eeeoeeesee
eoee


eoe
eooeeeeeeeee
eeeeeeeeeeeeeee
eee
eeeeeee
eeeeeeee
see
eoeeeeeeoee
eeeee
e

eee
eeeteeI
e'eeeeeeee
eee
eee
o
eee
eeee

oe
eeeeeee
ee
eeeeeoeeeooe


e
eeeeeeeoeeeeooeeeee

ee
eeeee


e


eoeoeheeeheeeeeeee
e

eeeee
eee
ee
ee

e

eeeee
ee
eee
eoeeee


----- diversity: 0.5
----- Generating with seed: "Another night I'm waiting for you  
Wait"
Another night I'm 

In [8]:
# sentence 7

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Back track, listen, stop that  \nBefore e'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Back track, listen, stop that  
Before e"
Back track, listen, stop that  
Before eeeeneee

eeee

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


e
oe
ao
eee
eeeoe
eee
e
e

eeeoeee
eeeee
eoeeeeteeoe

eooeeeeeeee
aeeeeeeeeeeoaeo'oeee
oee

oeeeo
oe
eee
eeeeeeeee

e'eseee
eeeeete
eeoeeeeeeeeee
eeeeeeee
eeeeooee

eeee'
eeeeeo
ee

oeeeeo
oe
eoeee
o
eee'se
eeeoee
ee
oooeeeteo
oeeeeeeoee

eeoe
ae
eeee
eeee

eohe
eeeee
eeehe
o
eeoeoeeoeee

ee
eoooe
eeeeeeeoeeeeeee
eo
eee
ett
eeeeooeee
e
e
eeeeeeo

e
eee
'ee
e

eh
oeeoteee
eee
e
oeeeee
oeeeeeeeeeeeoheeeete
eooeeeoeee
oeeeeeeeoee

eeooe
eee

ee

ee
e
e
eeeeeeeeoeeeeeee
e

eeeeeo

oeeeeo
eee
oee
ee

eee

eee
eeeeeeeeeeee

e

e
eeeeeeeeeeeee
eoe

eeoeee
e
ee
ee
eee
e
eoeeeeoee
o
e
eeeteeee

oee
eee

eeeeoe
eoeoeee




eeeeeoeeeee

eee
eee
eeooee

ooooeeeeeoee
eeee
h
oeoeoe
oeeoe

oeeo
e

e
ee

eeeeoe
eeeeeoeeeeoee
eeeeeeeeeeeeooo
e
ee
eneeeee
ee

ee

e

eeoeoeeoeeeeeeeteeeeee
oteee
e
oeeeeeeeo
eeeeeeetee
eeeeee
eeee

eo
e
e'eeeoteeeeeooe
eeeee
he
eee
ee
eee
e

eeoeoeto
eeeeoeooee

----- diversity: 0.5
----- Generating with seed: "Back track, listen, stop that  
Before e"
Back track, listen,

In [9]:
# sentence 8

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'How can a non-musician discuss the futur'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "How can a non-musician discuss the futur"
How can a non-musician discuss the futurheee
eeeeeeeo

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log



e
eeeeeeee
te
eeeoeeeoe
eeeet
ee
eeeeeeee
e
eee
a
e
e
eee
e

ee


oeeo
eeeeeeeeteeoe
eoeeeeeeeeeeeoo
eoeoee
eooheeeeeeeeeeeee
eee'eeeeeo
e
eeeeee
eeeeeee
eeeeeeehe
eeeeeeo
oee

eeeeeeeeeeee

eeeeeeeeeoeh
e

eeeo
e
seeee
oeeeeeeee

eee
eeee

e
eeo
e
ee
eeeoeee
eeeeeeeee
ee
e

e
eeeeeeeeeeeeeeeeeeeeeeeeeee
eoe
eoeee
eeeeeoe

eoe
eee

e
eoe

ee

eeee
ne

e

eee
eeeteoeeeeeee
e

eeoeeeoee
ee
eeeeeeeeeeee
eo
eeee
e
eeo
e
eeeeoeee

e
e
eeee


e
e



eeoeeeoeeee
eee
eeeneeeeeeee

eee

ee
eoo
oeteeeee
eeeee

e

eeee
eeeeeeeoee

Ieeeeeeeoe
eeeeeeeoee
e
eeoeeeeeeee
e
ote
e

eeeeeaoe
eeeeoeeeeeeeeeoeeee
eeeeeee
eooee
e
eeeeeeoee
eeeo
ee
eeeeeeeo
eeeo
eeee
enteeaeeeeoeeeeoeooo'eee
ee
eeee't
e
e
eoeeooee
eeeee
eeeoeooeeoe
e
ee
eee
e
eoe
eoeoe
eeeeoeeee
eeeeeeee
e
ee

oeeeooeeee

eteeee
eee
o

eee
teo
e
ee
eteeoeoeoee'eoee
eeeeeooeeeseoeeoeeeteee

oeee
eeeee
ee
eeeeee
o
eeeeee
eeee'eeeee

----- diversity: 0.5
----- Generating with seed: "How can a non-musician discuss the futur"
How can a non-music

In [10]:
# sentence 9

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Hi out there. This is Shaun Goldberg and'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Hi out there. This is Shaun Goldberg and"
Hi out there. This is Shaun Goldberg and
e


eeoee
e

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log



ae
eh
eeoo
ee
eo'eeeee
eeeeeo
eeee
eeeeteee
eeoeoee
eoeeeeeeeeeoee
e
e
seeeeeeeeeee


eeeee
e
e
oooeo
eeheeeeee

eeeeo'eeeeoe


eee
eeeee
eeeo
ee
eeeeoeeeee
e
ee
eee
eeeeeeeeeee
ee
eoeeeeeeeo
eeee
teeeoeoeeoeeeee

eeeeee

e
ee
eeeeoeoee
eoe
eeeaeeeeeoeteeeeeeoeeeoeee
oee

eee
eee
naeeeoeeeeeeoe
ee
e
eeeeeeeeee

eooeee
eee


et
'ee
o


e
eeeeeoeeeeeee
eeeeeoeeee
oeeeoe
oeeeeeeee
oee
eeee
teo
ee
e
eeeee
eee
e

eeee

ooeeeeeeeoaoeeeeeo
eeee
oee
eeeeeeeee
eee
ee
ee
oeeeooee

e
eee

eeeeeeeo
oe
ee

eeeeeeoo
ee
eeo
heeeeteeeeeeeeee
eo

eeee
e
t
eee

eoe


'ee
ee
eeee


oeeeooe
eoe
eoeeeoe


eeeoeee

o
ee
eooee
eeeeoo


eeeeaeee


e
e
eeoeeoore

eee

ae

eoeeeoeeeeeee
eeee
eeoeee
eeeeeeeeeete


eee
eeeo
eoee
e
eee
eee
e
eo


eeeeeeeeeee


e
eoeeoseeeeeo
eee
o
eeeeeo
eeheeeeteoe
e

eee
eeeeee
teeeeeeeo
e
e
eoeeeeoeee
e
eeeeeeoeeeooeeoneeeeeooeeeoeeeeeeeoseeeee


eee
teeeeeeoeeeee

e

----- diversity: 0.5
----- Generating with seed: "Hi out there. This is Shaun Goldberg and"
Hi out there. This

In [11]:
# sentence 10

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Whose eyes in my behind  \nI don't recogn"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Whose eyes in my behind  
I don't recogn"
Whose eyes in my behind  
I don't recogneeeeeeee
oee

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


eeea
oeeeeeee
eeee
oeeeeeeee
eo
eeeetee

e
ee
eeeeeeeeoe
e
e
eeeooeeee
eoeo

e
eeeee


e
ee

eteeeeoeeee
eeeee
eeeeeeeteeeooeeee
ee
e
oeoeoeoeeoeeoe
e
eoeeteo
eeeeeoa
ee
e
ee
e
e
ee'een
e

eoee
eeoeeoeee
ee
eeteeeeeeeeeeee
ee
e

ee
eeeee

eeoeoet
'eoe
eeeeeeeeeeoeeeeee
oeeeeeee
e

eo
eeeoe
eeeeee
eeo
eeeeeeeeoeeeeoe
eee
ee
eoeeee
eeeeoo
eeo
aoo

eeee
eaee

ee
oe
eeeeeee
ooeae
ee
ot

eeoeeeee
eee
ee
eee

ee
a
eeeeee

oe


e
ateeoetee
eteee
eeeeeeee
oeeeteo
e

ooe
eeee
eeee


e
e
ee

eeeeeeeeeee
ee
ee
ee
oeee
ee
eee
'eeeeeoeeeoe
eee
eeeteee

ooeeeeee


eeeo
eeeoeeeeee
oeeeteeeeeeeeeet
eeee
ee
eteeeo
e

eoeeeeeeee'eo
eeeeeeeee
eoeeteeee

e
e

eee
eoeee
eeo
e

oe
eo
o


eeeeaeeeeeeeeo
eoeeeoeeeeeoee
eeeeoeee
e
eoo
eeoeeeoeeeeeeeeoheeh
e
eo
eoeeeeo
eeo
oeoeeoeeee
ee

eeeeoeeoe
ee
eoeeeeeeo
eeeoe
eo

ee
et
eeeo
ese
ee

eae
eeoeeeeee
eeee
ee'eee

e
e
eoe
etee
eeeoeee
ee
eeeoe


eeee

----- diversity: 0.5
----- Generating with seed: "Whose eyes in my behind  
I don't recogn"
Whose eyes in my b